In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

import warnings
warnings.filterwarnings("ignore") 

In [2]:
# Read the data file

SolarSurvey = pd.read_csv('SolarSurvey.csv')
SolarSurvey.head()

,Household_ID,City,Latitude,House_Type,Roof_Type,Roof_Pitch,Roof_Azimuth,Floors,Financed,Year,Panel_Capacity,Shading,Generation
0,A553C5784D,Brisbane,NaN,NaN,Steel,10.0,21.0,Double,No,2019,5040,None,10163.796
1,1A12B988A0,Brisbane,-27.217398,Timber Frame,Tile,35.0,50.0,Single,Yes,2011,5880,NaN,10876.910
2,D23C0BBB05,Brisbane,-27.252449,Double Brick,NaN,20.0,272.0,Single,Yes,2018,5040,NaN,9400.415
3,E67A638AD3,Brisbane,-27.042364,Timber Frame,Steel,35.0,247.0,NaN,Yes,2012,5040,None,7747.349
4,7F266FCAD1,Brisbane,-27.432734,Other,Steel,10.0,39.0,Double,Yes,2012,5880,None,10987.674


In [3]:
SolarSurvey

,Household_ID,City,Latitude,House_Type,Roof_Type,Roof_Pitch,Roof_Azimuth,Floors,Financed,Year,Panel_Capacity,Shading,Generation
0,A553C5784D,Brisbane,NaN,NaN,Steel,10.0,21.0,Double,No,2019,5040,None,10163.796
1,1A12B988A0,Brisbane,-27.217398,Timber Frame,Tile,35.0,50.0,Single,Yes,2011,5880,NaN,10876.910
2,D23C0BBB05,Brisbane,-27.252449,Double Brick,NaN,20.0,272.0,Single,Yes,2018,5040,NaN,9400.415
3,E67A638AD3,Brisbane,-27.042364,Timber Frame,Steel,35.0,247.0,NaN,Yes,2012,5040,None,7747.349
4,7F266FCAD1,Brisbane,-27.432734,Other,Steel,10.0,39.0,Double,Yes,2012,5880,None,10987.674
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,3C1434F399,Melbourne,-37.741982,Other,Tile,30.0,132.0,Single,Yes,2011,7560,Significant,4252.178
2996,BAC8883CA0,Melbourne,-38.008012,Other,Tile,10.0,57.0,Double,Yes,2020,7140,Significant,5684.912
2997,9192858124,Melbourne,-38.342210,Timber Frame,Tile,22.5,352.0,Single,Yes,2017,7140,None,12026.470
2998,C2186B8C95,Melbourne,-37.579522,Timber Frame,Tile,5.0,115.0,Single,No,2014,6300,None,8902.884


In [4]:
SolarSurvey.columns

Index(['Household_ID', 'City', 'Latitude', 'House_Type', 'Roof_Type',
       'Roof_Pitch', 'Roof_Azimuth', 'Floors', 'Financed', 'Year',
       'Panel_Capacity', 'Shading', 'Generation'],
      dtype='object')

In [5]:
SolarSurvey_selected = SolarSurvey.drop('Household_ID',axis = 1)
SolarSurvey_selected.shape

(3000, 12)

In [6]:
SolarSurvey_selected.isna().sum()

City                0
Latitude           70
House_Type        182
Roof_Type         339
Roof_Pitch         28
Roof_Azimuth       84
Floors            269
Financed            0
Year                0
Panel_Capacity      0
Shading            96
Generation          0
dtype: int64

In [9]:
SolarSurvey_selected['Latitude'].fillna(SolarSurvey_selected['Latitude'].mean(),inplace = True)
SolarSurvey_selected['Roof_Azimuth'].fillna(SolarSurvey_selected['Roof_Azimuth'].mean(),inplace = True)
SolarSurvey_selected['Roof_Pitch'].fillna(SolarSurvey_selected['Roof_Pitch'].mode()[0],inplace = True)

SolarSurvey_selected['Roof_Type'].fillna('Other',inplace = True)
SolarSurvey_selected['House_Type'].fillna('Other',inplace = True)
SolarSurvey_selected['Floors'].fillna('Other',inplace = True)
SolarSurvey_selected['Shading'].fillna('Partial',inplace = True)

In [78]:
SolarSurvey_selected.isna().sum()

City                        0
Latitude                    0
House_Type                  0
Roof_Type                   0
Roof_Pitch                  0
Floors                      0
Financed                    0
Year                        0
Panel_Capacity              0
Shading                     0
Generation                  0
Roof_Azimuth_transformed    0
dtype: int64

In [80]:
SolarSurvey_selected = SolarSurvey_selected.dropna()

In [11]:
SolarSurvey_selected = SolarSurvey_selected[SolarSurvey_selected['Generation'] >= 0]

SolarSurvey_selected['Roof_Azimuth_transformed'] = SolarSurvey_selected['Roof_Azimuth']

for alpha in SolarSurvey_selected.index:
    if SolarSurvey_selected.loc[alpha, 'Roof_Azimuth'] > 180:
        SolarSurvey_selected.loc[alpha, 'Roof_Azimuth_transformed'] = 360 - SolarSurvey_selected.loc[alpha, 'Roof_Azimuth']
        
SolarSurvey_selected = SolarSurvey_selected.drop('Roof_Azimuth',axis = 1)

KeyError: 'Roof_Azimuth'

In [82]:
SolarSurvey_selected.shape

(2985, 12)

In [83]:
SolarSurvey_encoded = SolarSurvey.copy()

In [84]:
print(SolarSurvey_encoded['Shading'].value_counts())

None           2215
Partial         561
Significant     128
Name: Shading, dtype: int64


In [86]:
keys = {
    'None': 1,
    'Partial': 0.75,
    'Significant':0.5
}
SolarSurvey_encoded['Shading'] = SolarSurvey_encoded['Shading'].replace(keys)

In [87]:
print(SolarSurvey_encoded['City'].value_counts())
print('-------------------')
print(SolarSurvey_encoded['House_Type'].value_counts())
print('-------------------')
print(SolarSurvey_encoded['Roof_Type'].value_counts())
print('-------------------')
print(SolarSurvey_encoded['Floors'].value_counts())
print('-------------------')
print(SolarSurvey_encoded['Financed'].value_counts())


Brisbane     1000
Sydney       1000
Melbourne    1000
Name: City, dtype: int64
-------------------
Timber Frame    1988
Double Brick     559
Other            258
Weatherboard      13
Name: House_Type, dtype: int64
-------------------
Tile         1358
Steel        1010
Other         259
Colorbond      34
Name: Roof_Type, dtype: int64
-------------------
Single    1180
Double    1118
Other      433
Name: Floors, dtype: int64
-------------------
Yes    2049
No      951
Name: Financed, dtype: int64


In [69]:
SolarSurvey_encoded = pd.get_dummies(SolarSurvey_encoded, drop_first = True)

print(SolarSurvey_encoded.shape)
SolarSurvey_encoded

(3000, 3017)


,Latitude,Roof_Pitch,Roof_Azimuth,Year,Panel_Capacity,Shading,Generation,Household_ID_003AD50119,Household_ID_0048ABE935,Household_ID_005C6F25D4,...,City_Sydney,House_Type_Other,House_Type_Timber Frame,House_Type_Weatherboard,Roof_Type_Other,Roof_Type_Steel,Roof_Type_Tile,Floors_Other,Floors_Single,Financed_Yes
0,NaN,10.0,21.0,2019,5040,1.0,10163.796,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,-27.217398,35.0,50.0,2011,5880,NaN,10876.910,0,0,0,...,0,0,1,0,0,0,1,0,1,1
2,-27.252449,20.0,272.0,2018,5040,NaN,9400.415,0,0,0,...,0,0,0,0,0,0,0,0,1,1
3,-27.042364,35.0,247.0,2012,5040,1.0,7747.349,0,0,0,...,0,0,1,0,0,1,0,0,0,1
4,-27.432734,10.0,39.0,2012,5880,1.0,10987.674,0,0,0,...,0,1,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,-37.741982,30.0,132.0,2011,7560,0.5,4252.178,0,0,0,...,0,1,0,0,0,0,1,0,1,1
2996,-38.008012,10.0,57.0,2020,7140,0.5,5684.912,0,0,0,...,0,1,0,0,0,0,1,0,0,1
2997,-38.342210,22.5,352.0,2017,7140,1.0,12026.470,0,0,0,...,0,0,1,0,0,0,1,0,1,1
2998,-37.579522,5.0,115.0,2014,6300,1.0,8902.884,0,0,0,...,0,0,1,0,0,0,1,0,1,0


In [70]:
SolarSurvey_encoded['Year'] = 2022 -SolarSurvey_encoded['Year']
SolarSurvey_encoded['Year']

0        3
1       11
2        4
3       10
4       10
        ..
2995    11
2996     2
2997     5
2998     8
2999     1
Name: Year, Length: 3000, dtype: int64

In [88]:
SolarSurvey_rescaled = SolarSurvey_encoded.copy()

rescaled_col = ['Latitude','Roof_Pitch','Roof_Azimuth_transformed','Year','Panel_Capacity']
for z in rescaled_col:
    minx = min(SolarSurvey_rescaled[z])
    maxx = max(SolarSurvey_rescaled[z])
    SolarSurvey_rescaled[z] = (SolarSurvey_rescaled[z]-minx)/(maxx-minx)

KeyError: 'Roof_Azimuth_transformed'

In [91]:
from sklearn.model_selection import train_test_split

alpha_SolarSurvey = SolarSurvey_rescaled.drop('Generation', axis = 1)
beta_SolarSurvey = SolarSurvey_rescaled['Generation']

alpha_train_all, alpha_test, beta_train_all, beta_test = train_test_split(alpha_SolarSurvey, beta_SolarSurvey,
                                                               test_size = 0.2,
                                                               random_state = 0)
alpha_train, alpha_vali, beta_train, bate_vali = train_test_split(alpha_train_all, beta_train_all,
                                                                test_size = 0.2,
                                                               random_state = 0)
print(alpha_SolarSurvey.shape)
print(alpha_train.shape)
print(alpha_vali.shape)
print(alpha_test.shape)

(3000, 12)
(1920, 12)
(480, 12)
(600, 12)


## Question 1
### (a)

## Question 2

## Question 3
### (a)

In [58]:
# === End of Assignment ===